---
title: "Catboost Modelling"
author: "Alex, Rachel, Vlatko, Malte"
date: "2024-06-20"
format: html
execute: 
  cache: true
  echo: false
  error: true
jupyter: python3
editor:
  render-on-save: true
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
from matplotlib.patches import Patch
import seaborn as sns
import datetime as dt

from sklearn.impute import SimpleImputer, KNNImputer 
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error

from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold
from statsmodels.tsa.stattools import adfuller

from catboost import CatBoostRegressor
import catboost as cb
from xgboost import XGBRegressor
import optuna 

In [ ]:
%run functions_model.py
%run functions_vis.py

In [ ]:
! jupyter nbextension enable widgetsnbextension --py

In [ ]:
pd.set_option("display.max_columns", None)

# Loading the dataset

In [ ]:
d = pd.read_csv("data/train_df.csv")
d_test = pd.read_csv("data/test_df.csv")

In [ ]:
d['date'] = pd.to_datetime(d['date'])
d_test['date'] = pd.to_datetime(d_test['date'])

# Creating validation dataset

In [ ]:
train, val = create_val_set(d)

# Modelling

## Catboost

### Selecting features

In [ ]:
date = ["date"]

catfeat = ["store_name","item_category","hol_pub","hol_school","weekday","day","month","year","week_year","nye","valentines_day","halloween", "street_market","public_space","box_deal"]

numfeat = ["days_back","temperature_2m_mean","sunshine_duration","precipitation_hours"]

lag = ["lag1","lag2"]

In [ ]:
x_train = train[date + catfeat + numfeat + lag]
x_train = x_train.set_index("date")
x_train_daily = x_train[(x_train["item_category"] == "daily total")].drop("item_category", axis = 1)
y_train = train['total_amount']
y_train_daily = train[(train["item_category"] == "daily total")]['total_amount']

x_val = val[date + catfeat + numfeat + lag]
x_val = x_val.set_index("date")
x_val_daily = x_val[(x_val["item_category"] == "daily total")].drop("item_category", axis = 1)
y_val = val['total_amount']
y_val_daily = val[(val["item_category"] == "daily total") ]['total_amount']

x_test = d_test[date + catfeat + numfeat]
x_test = x_test.set_index("date")
x_test_daily = x_test[(x_test["item_category"] == "daily total")].drop("item_category", axis = 1)
y_test = d_test['total_amount']
y_test_daily = d_test[(d_test["item_category"] == "daily total")]['total_amount']

In [ ]:
catfeat.remove("item_category")

### Convert holiday features to integer

In [ ]:
x_train["hol_pub"] = x_train["hol_pub"].apply(np.int64)
x_train["hol_school"] = x_train["hol_school"].apply(np.int64)
x_train_daily["hol_pub"] = x_train_daily["hol_pub"].apply(np.int64)
x_train_daily["hol_school"] = x_train_daily["hol_school"].apply(np.int64)

x_val["hol_pub"] = x_val["hol_pub"].apply(np.int64)
x_val["hol_school"] = x_val["hol_school"].apply(np.int64)
x_val_daily["hol_pub"] = x_val_daily["hol_pub"].apply(np.int64)
x_val_daily["hol_school"] = x_val_daily["hol_school"].apply(np.int64)

x_test["hol_pub"] = x_test["hol_pub"].apply(np.int64)
x_test["hol_school"] = x_test["hol_school"].apply(np.int64)
x_test_daily["hol_pub"] = x_test_daily["hol_pub"].apply(np.int64)
x_test_daily["hol_school"] = x_test_daily["hol_school"].apply(np.int64)

In [ ]:
x_train_daily_totalamount = pd.concat([x_train_daily.reset_index(), y_train_daily.reset_index(drop= True)], axis = 1)

x_test_daily_totalamount = pd.concat([x_test_daily.reset_index(), y_test_daily.reset_index(drop= True)], axis = 1)

### Custom CV split

In [ ]:
def create_train_validation_folds(df):
    last_val_date = df.date.max()
    cv = []
    for n in range(5):
        first_val_date = last_val_date - dt.timedelta(days=6)
        train_index = df[df.date<first_val_date].index
        val_index = df[(df.date>=first_val_date) & (df.date<=last_val_date)].index
        cv.append((train_index,val_index))
        last_val_date = first_val_date - dt.timedelta(days=1)
    cv = cv[::-1]
    return cv

In [ ]:
create_train_validation_folds(x_train_daily_totalamount)

In [ ]:
def create_train_validation_dataset(df):
    last_val_date = df.date.max()
    cv = []
    for n in range(5):
        first_val_date = last_val_date - dt.timedelta(days=6)
        train_fold = df[df.date<first_val_date]
        val_fold = df[(df.date>=first_val_date) & (df.date<=last_val_date)]
        cv.append((train_fold,val_fold))
        last_val_date = first_val_date - dt.timedelta(days=1)
    cv = cv[::-1]
    return cv

In [ ]:
for i in range(5):
  globals()[f"train_{i}"], globals()[f"val_{i}"] = create_train_validation_dataset(x_train_daily_totalamount)[i]

In [ ]:
val_4.head()

In [ ]:
train_val_datasets = [
    ("0", train_0, val_0),
    ("1", train_1, val_1),
    ("2", train_2, val_2),
    ("3", train_3, val_3),
    ("4", train_4, val_4)
]

In [ ]:
for set, train, val:

### Hyperparameter Tuning


In [ ]:
grid = {
    'n_estimators': np.arange(1000,5001,1),
    "learning_rate": np.linspace(1e-3, 0.1),
    "l2_leaf_reg": np.linspace(2, 30),
    "depth":np.arange(3,21,1),
    "subsample": np.linspace(0.05, 1.0),
    "colsample_bylevel": np.linspace(0.05, 1.0),
    "min_data_in_leaf": np.arange(10, 101, 1),
    #"min_child_weight": np.arange(1, 301, 1),
    "early_stopping_rounds": np.arange(20, 81, 1)}

In [ ]:
model = cb.CatBoostRegressor(cat_features=catfeat)

In [ ]:
randomized_search_result = model.randomized_search(grid,
                                                  # n_iter=30,
                                                   X=x_train_daily,
                                                   y=y_train_daily,
                                                   cv = create_train_validation_folds(x_train_daily_totalamount),
                                                   plot=True,
                                                   verbose = 500,
                                                   calc_cv_statistics=True)

In [ ]:
pd.DataFrame(randomized_search_result.get("cv_results"))

In [ ]:
randomized_search_result.get("params")

In [ ]:
#def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 2, 30),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "min_child_weight": trial.suggest_int(1, 300),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 50, 80)
    }

    model = CatBoostRegressor(**params, silent=True, allow_writing_files = False, cat_features = catfeatures)

    model_cv = optuna.integration.OptunaSearchCV(estimator = model, param_distributions = params, cv = create_train_validation_folds(x_train_daily_totalamount), n_jobs = -1, n_trials = 30)
    
    #model.fit(x_train_daily, y_train_daily, eval_set = (create_train_validation_folds_cat(x_train_daily_totalamount)), cat_features = catfeat, verbose = 1000, plot = False)

    # y_val_pred_daily = model_cv.predict(x_val_daily)
    
    #rmse = mean_squared_error(y_val_daily, y_val_pred_daily, squared=False)

    #return rmse

#study = optuna.create_study(direction='minimize')
#study.optimize(objective, n_trials=30)

In [ ]:
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

In [ ]:
# Best model

model = CatBoostRegressor(n_estimators = 2620, learning_rate = 0.0487,
 depth = 5, subsample = 0.21, colsample_bylevel = 0.2, min_data_in_leaf = 21, early_stopping_rounds = 69, cat_features = catfeat, random_state = 123, allow_writing_files = True)

model.fit(x_train_daily, y_train_daily, eval_set=(x_val_daily, y_val_daily), verbose = 500, plot = True)

### Feature importances

In [ ]:
model.get_feature_importance(prettified=True).plot(x = "Feature Id", y = "Importances", kind = "bar")

### Evaluation metrics train and validation set

In [ ]:
y_train_pred = model.predict(x_train_daily)
print(f"R-squared train: {round(r2_score(y_train_daily, y_train_pred),6)}")
print(f"MAPE train: {round(100*mean_absolute_percentage_error(y_train_daily, y_train_pred),2)}\n")

y_val_pred = model.predict(x_val_daily)
print(f"R-squared validation: {round(r2_score(y_val_daily, y_val_pred),6)}")
print(f"MAPE validation: {round(100*mean_absolute_percentage_error(y_val_daily, y_val_pred),2)}")

# Test dataset

In [ ]:
x_test_daily_totalamount, y_test_pred =  pred_test(train = x_train_daily_totalamount, test = x_test_daily_totalamount, model = model, numfeat = numfeat, catfeat = catfeat)

### Residuals by store

In [ ]:
difference_df = diff_overview(data = x_test_daily_totalamount, pred = y_test_pred, stores = all)

### Residual Plot Analysis 

In [ ]:
sns.scatterplot(data = difference_df, x = "Predicted", y = "Stand_resid")

plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel("Predicted Sales (Daily total)")
plt.ylabel("Standardized Residuals")

plt.show()

### Evaluation metrics train and test set

In [ ]:
fit_overview(ytrain = y_train_daily, ytrainpred = y_train_pred, ytest = y_test_daily, ytestpred = y_test_pred)

In [ ]:
mape_stores(x_test_daily_totalamount, y_test_pred)

### Visualisation of predictions

In [ ]:
df_predicted = pd.concat(
    [
    difference_df[["Date","Store name","Observed","Predicted"]].rename(columns = {"Date":"date","Store name":"store_name","Observed":"total_amount"}),
    d[(d["date"] >= pd.to_datetime("2024-05-01")) & (d["item_category"] == "daily total")][["date","store_name","total_amount"]]
    ]
    )

In [ ]:
ts_predicted(df_predicted)